### start

In [19]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.workflow_orchestrator as wwo
from utils import ConfigError
import utils as u

# reload all modules
modules = [
    wm, wwo,
    u
]

# load all configs
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


u.export_code(
    code_directories=[
        # 'etls',
        'sagemaker_wallets',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[06/Jul/25 17:12:38] INFO [utils.export_code:1542] Consolidation complete. All files are saved in temp/consolidated_code.py
[06/Jul/25 17:12:38] MILESTONE [2564028623.<module>:108] Good morning, let's get to work


# Initial ETLs

# Code begins

#### initiate orchestrator

In [4]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


workflow_orch = wwo.WalletWorkflowOrchestrator(sage_wallets_config)

#### load and upload data

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffixes = [
    '231107',
    '240306'
]
workflow_orch.load_training_data(date_suffixes)

In [ ]:
# Upload and retrieve URIs
workflow_orch.upload_training_data(overwrite_existing=False)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

### retrieve uris

In [20]:
date_suffixes = [
    '231107',
    # '240306'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

{'231107': {'x_train': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_train.csv',
  'y_train': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_train.csv',
  'x_test': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_test.csv',
  'y_test': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_test.csv',
  'x_eval': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_eval.csv',
  'y_eval': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_eval.csv',
  'x_val': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_val.csv',
  'y_val': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_val.csv'}}

### dda 872 devspace

In [23]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, s3_uris)




In [24]:
modeler.train_model()

[06/Jul/25 17:13:50] INFO [wallet_modeler.train_model:56] Starting SageMaker XGBoost training
[06/Jul/25 17:13:50] INFO [image_uris._processor:530] Ignoring unnecessary instance type: None.
[06/Jul/25 17:13:50] INFO [wallet_modeler.train_model:101] Launching training job: wallet-xgb-231107-171350
[06/Jul/25 17:13:50] INFO [telemetry_logging.wrapper:91] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[06/Jul/25 17:13:50] INFO [session.submit:1053] Creating training-job with name: wallet-xgb-231107-171350


2025-07-07 00:13:51 Starting - Starting the training job...
2025-07-07 00:14:26 Downloading - Downloading input data......
2025-07-07 00:15:11 Downloading - Downloading the training image......
2025-07-07 00:16:17 Training - Training image download completed. Training in progress.
2025-07-07 00:16:17 Uploading - Uploading generated training model/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-07-07 00:16:12.497 ip-10-0-236-180.us-west-2.compute.internal:8 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-07-07 00:16:12.527 ip-10-0-236-180.us-west-2.compute.internal:8 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-07-07:00:16:12:INFO] Imported framework sagemaker_xgboost_container.training
[2025-07-07:00:16:12:INFO] Failed to parse hyperpara

[06/Jul/25 17:17:13] INFO [wallet_modeler.train_model:116] Training completed. Model stored at: s3://wallet-training-data/sagemaker-models/wallet-xgb-231107-171350/output/model.tar.gz


{'model_uri': 's3://wallet-training-data/sagemaker-models/wallet-xgb-231107-171350/output/model.tar.gz',
 'training_job_name': 'wallet-xgb-231107-171350'}

In [ ]:
workflow_orch.training_data['x_train'].shape